In [64]:
from datetime import datetime, timedelta

def is_leap_year(year):
    """Check if a given year is a leap year."""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def calculate_vibration_dates(start_date, degrees):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    year = start_date.year
    total_days = 366 if is_leap_year(year) else 365 
    
    days_per_degree = total_days / 360
    
    vibration_dates = {}
    
    for degree in degrees:
        days_to_add = int(round(degree * days_per_degree))
        
        vibration_date = start_date + timedelta(days=days_to_add)
        
        while vibration_date.year > year:
            vibration_date -= timedelta(days=total_days)
        while vibration_date.year < year:
            vibration_date += timedelta(days=total_days)
        
        vibration_dates[degree] = vibration_date.strftime("%Y-%m-%d")
    
    return vibration_dates

# Example usage
start_date = "2024-04-19"  
degrees = [0, 90, 180, 270, 120, 240, 144, 216]  
vibration_dates = calculate_vibration_dates(start_date, degrees)

for degree, date in vibration_dates.items():
    print(f"Degree {degree}: {date}")


Degree 0: 2024-04-19
Degree 90: 2024-07-20
Degree 180: 2024-10-19
Degree 270: 2024-01-18
Degree 120: 2024-08-19
Degree 240: 2024-12-19
Degree 144: 2024-09-12
Degree 216: 2024-11-25


In [66]:
import pandas as pd
import numpy as np
import yfinance as yf
from enum import Enum
import plotly.graph_objects as go
from datetime import datetime,timedelta

class GannSwing:
    '''
    Class to perform Gann swing calculations
    '''
    def __init__(self, bars: pd.DataFrame):
        '''
        Parameters:
        - (required) a pandas dataframe containing OHLC data
        '''
        self.bars = bars
        self.__validate_bars(bars)

    def __validate_bars(self, bars):
        if not isinstance(self.bars, pd.DataFrame):
            raise TypeError('bars should be a Pandas dataframe')
        mandatory_columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        columns = list(bars.columns)
        for i in mandatory_columns:
            if i not in columns:
                raise IndexError('bars is missing a column named "%s"' % i)

    class Trend(Enum):
        UNKNOWN = np.nan
        UP = 'Up'
        DOWN = 'Down'

    def calculate_swings(self, swing_days: int = 1, inside_down: bool = False, ignore_threshold: int = 0, use_close_of_outside_bar: bool = False) -> pd.DataFrame:
        self.swing_days = swing_days
        self.inside_down = inside_down
        self.ignore_threshold = ignore_threshold
        self.use_close_of_outside_bar = use_close_of_outside_bar

        # Initialize list to store swing points
        swing_points = []

        # Calculate swing points by checking for local maxima and minima
        for i in range(swing_days, len(self.bars) - swing_days):
            high = self.bars['High'][i]
            low = self.bars['Low'][i]
            is_swing_high = all(self.bars['High'][i] > self.bars['High'][i - j] for j in range(1, swing_days + 1)) and \
                            all(self.bars['High'][i] > self.bars['High'][i + j] for j in range(1, swing_days + 1))
            is_swing_low = all(self.bars['Low'][i] < self.bars['Low'][i - j] for j in range(1, swing_days + 1)) and \
                           all(self.bars['Low'][i] < self.bars['Low'][i + j] for j in range(1, swing_days + 1))
            
            if is_swing_high or is_swing_low:
                swing_points.append({
                    'Timestamp': self.bars['Timestamp'][i],
                    'SwingPrice': high if is_swing_high else low,
                    'Trend': 'Up' if is_swing_high else 'Down'
                })

        # Convert results to a DataFrame
        results = pd.DataFrame(swing_points)
        return results

    def visualise(self):
        '''
        Draw an OHLC chart of the bars data. If swings have been calculated, overlay them
        on top of the OHLC chart.
        '''
        hovertext = []
        for i in range(len(self.bars['Open'])):
            hovertext.append('Open: ' + str(self.bars['Open'][i]) + '<br>High: ' + str(self.bars['High'][i]) +
                             '<br>Low: ' + str(self.bars['Low'][i]) + '<br>Close: ' + str(self.bars['Close'][i]))

        fig = go.Figure(data=go.Ohlc(x=self.bars['Timestamp'],
                                     open=self.bars['Open'],
                                     high=self.bars['High'],
                                     low=self.bars['Low'],
                                     close=self.bars['Close']),
                        )
        fig.update(layout_xaxis_rangeslider_visible=False)

        # If swings have been calculated, overlay them
        swings = self.calculate_swings()
        fig.add_trace(go.Scatter(x=swings['Timestamp'],
                                 y=swings['SwingPrice'],
                                 mode='markers',
                                 marker=dict(color='blue', size=8),
                                 name='Swing Points'))
        fig.show()

# Function to get data from Yahoo Finance
def get_yahoo_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.reset_index(inplace=True)
    data = data.rename(columns={'Date': 'Timestamp'})
    return data[['Timestamp', 'Open', 'High', 'Low', 'Close']]

# Main function to run Gann swing analysis
def calculate_swing_points(ticker, start_date, end_date):
    # Get data from Yahoo Finance
    bars = get_yahoo_data(ticker, start_date, end_date)
    
    # Initialize GannSwing class with data
    gs = GannSwing(bars)
    print(gs)
    
    swing_points = gs.calculate_swings(swing_days=5)
    
    # Display results
    # print("Swing Points:")
    # print(swing_points)

    # Visualize results
    # gs.visualise()
    
    return swing_points

# Example usage
sp=calculate_swing_points(ticker='AAPL', start_date='2024-01-01', end_date='2024-03-31')


[*********************100%***********************]  1 of 1 completed

In [ ]:
tickers ={'jpy': 'JPY%3DX',
        'eur': 'EURUSD%3DX',
        'gbp': 'GBPUSD%3DX',
        'franc': 'CHFUSD%3DX',
        'us100': 'NQ%3DF',
        'us30': 'YM%3DF',
        'us500': 'ES%3DF',
        'crude oil': 'CL%3DF',
        'gold': 'GC%3DF',
        'bitcoin': 'BTC-USD',
        'Apple': 'AAPL',
        'Microsoft': 'MSFT',
        'Google': 'GOOGL',
        'Amazon': 'AMZN',
        'NVIDIA': 'NVDA',
        'Berkshire': 'BRK-B',
        'Meta': 'META',
        'Tesla': 'TSLA',

    }
swing_points_list=[]
for ticker in tickers.keys():
    stock={}    

    sp=calculate_swing_points(ticker=tickers[ticker], start_date='2024-01-01', end_date='2024-03-31')
    print(ticker)
    stock = {
    'Ticker': [ticker],
    'Min_price_date': [sp.loc[sp['SwingPrice'].idxmin(), 'Timestamp']],
    'Max_price_date': [sp.loc[sp['SwingPrice'].idxmax(), 'Timestamp']]
    }
    swing_points_list.append(stock)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


jpy
eur


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


gbp
franc
us100
us30
us500
crude oil


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


gold
bitcoin
Apple
Microsoft
Google


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Amazon
NVIDIA
Berkshire
Meta
Tesla


In [ ]:
main_dates=[]
degrees = [0, 90, 180, 270, 120, 240, 144, 216]
for i in range(len(swing_points_list)):
    degree_dict = {}
    ticker = swing_points_list[i]['Ticker'][0]  

    for key in swing_points_list[i].keys():
        if key != 'Ticker': 
            start_date = swing_points_list[i][key][0]  
            start_date = start_date.strftime('%Y-%m-%d')  
            
            vibration_dates = calculate_vibration_dates(start_date, degrees)
            
            
            degree_dict = {
                'Ticker': ticker,
                f'{key}': start_date,  
                'vibration_dates': vibration_dates
            }
            
            # Append to main_dates
            main_dates.append(degree_dict)

# Display the result
print(main_dates)
            
            
           

[{'Ticker': 'jpy', 'Min_price_date': '2024-02-01', 'vibration_dates': {0: '2024-02-01', 90: '2024-05-03', 180: '2024-08-02', 270: '2024-11-01', 120: '2024-06-02', 240: '2024-10-02', 144: '2024-06-26', 216: '2024-09-08'}}, {'Ticker': 'jpy', 'Max_price_date': '2024-02-28', 'vibration_dates': {0: '2024-02-28', 90: '2024-05-30', 180: '2024-08-29', 270: '2024-11-28', 120: '2024-06-29', 240: '2024-10-29', 144: '2024-07-23', 216: '2024-10-05'}}, {'Ticker': 'eur', 'Min_price_date': '2024-02-14', 'vibration_dates': {0: '2024-02-14', 90: '2024-05-16', 180: '2024-08-15', 270: '2024-11-14', 120: '2024-06-15', 240: '2024-10-15', 144: '2024-07-09', 216: '2024-09-21'}}, {'Ticker': 'eur', 'Max_price_date': '2024-03-08', 'vibration_dates': {0: '2024-03-08', 90: '2024-06-08', 180: '2024-09-07', 270: '2024-12-07', 120: '2024-07-08', 240: '2024-11-07', 144: '2024-08-01', 216: '2024-10-14'}}, {'Ticker': 'gbp', 'Min_price_date': '2024-02-05', 'vibration_dates': {0: '2024-02-05', 90: '2024-05-07', 180: '2024

In [118]:
swing_points_list[0]['Min_price_date'][0]

Timestamp('2024-02-01 00:00:00')

In [130]:
for i in main_dates:
    print(i)

{'Ticker': 'jpy', 'Min_price_date': '2024-02-01', 'vibration_dates': {0: '2024-02-01', 90: '2024-05-03', 180: '2024-08-02', 270: '2024-11-01', 120: '2024-06-02', 240: '2024-10-02', 144: '2024-06-26', 216: '2024-09-08'}}
{'Ticker': 'jpy', 'Max_price_date': '2024-02-28', 'vibration_dates': {0: '2024-02-28', 90: '2024-05-30', 180: '2024-08-29', 270: '2024-11-28', 120: '2024-06-29', 240: '2024-10-29', 144: '2024-07-23', 216: '2024-10-05'}}
{'Ticker': 'eur', 'Min_price_date': '2024-02-14', 'vibration_dates': {0: '2024-02-14', 90: '2024-05-16', 180: '2024-08-15', 270: '2024-11-14', 120: '2024-06-15', 240: '2024-10-15', 144: '2024-07-09', 216: '2024-09-21'}}
{'Ticker': 'eur', 'Max_price_date': '2024-03-08', 'vibration_dates': {0: '2024-03-08', 90: '2024-06-08', 180: '2024-09-07', 270: '2024-12-07', 120: '2024-07-08', 240: '2024-11-07', 144: '2024-08-01', 216: '2024-10-14'}}
{'Ticker': 'gbp', 'Min_price_date': '2024-02-05', 'vibration_dates': {0: '2024-02-05', 90: '2024-05-07', 180: '2024-08-0

In [ ]:
from datetime import datetime

# Today's date
today_date = datetime.now().strftime('%Y-%m-%d')

# List to store results
today_list = []

# Iterate through the main_dates
for i in range(len(main_dates)):
    degree_dict = {}
    ticker = main_dates[i]['Ticker']
    match_dict = {}  

    # Iterate through the keys of the current dictionary in main_dates
    for key in main_dates[i].keys():
        # Check if the key is 'Min_price_date' or 'Max_price_date'
        if key == 'Min_price_date' or key == 'Max_price_date':
            vibration_dates = main_dates[i]['vibration_dates']
            for degree, date in vibration_dates.items():
                if date == today_date:
                    print(date)
                    print(degree)
                    print(key)
                    match_dict = {
                        'ticker': ticker,
                        'actual_date': main_dates[i][key],  
                        'degree': degree
                    }
                    today_list.append(match_dict)  
        else:
            continue

# Print the result
print(today_list)


2024-11-09
0
Min_price_date
[{'ticker': 'jpy', 'actual_date': '2024-02-01', 'degree': 0}]


In [159]:
main_dates[0]

{'Ticker': 'jpy',
 'Min_price_date': '2024-02-01',
 'vibration_dates': {0: '2024-11-09',
  90: '2024-05-03',
  180: '2024-08-02',
  270: '2024-11-01',
  120: '2024-06-02',
  240: '2024-10-02',
  144: '2024-06-26',
  216: '2024-09-08'}}

In [144]:
vibration_dates = main_dates[0]['vibration_dates']
    

2024-11-09
0


In [148]:
vibration_dates

{0: '2024-02-01',
 90: '2024-05-03',
 180: '2024-08-02',
 270: '2024-11-01',
 120: '2024-06-02',
 240: '2024-10-02',
 144: '2024-06-26',
 216: '2024-09-08'}

'2024-11-09'